In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Coding Challenge - 4").getOrCreate()

# Load lineitem table as a DataFrame
df_lineitem = spark.read.format("delta").load("/databricks-datasets/tpch/delta-001/lineitem")

# Display the DataFrame
df_lineitem.show(5)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|   REG AIR| even pinto beans...|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-05-31|  1994-06-20|   1994-06-22|            NONE|       FOB|detect 

In [0]:
#Select Specific Columns
df_selected = df_lineitem.select("l_orderkey", "l_quantity", "l_extendedprice")
df_selected.show(5)


+----------+----------+---------------+
|l_orderkey|l_quantity|l_extendedprice|
+----------+----------+---------------+
|  15997987|     50.00|       66516.00|
|  15997988|     49.00|       53460.96|
|  15997988|     37.00|       47738.88|
|  15997988|     13.00|       25036.44|
|  15997988|     29.00|       34744.32|
+----------+----------+---------------+
only showing top 5 rows



In [0]:
#Filter rows where the quantity is greater than 10:
df_filtered = df_lineitem.filter(df_lineitem.l_quantity > 10)
df_filtered.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|   REG AIR| even pinto beans...|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-05-31|  1994-06-20|   1994-06-22|            NONE|       FOB|detect 

In [0]:
from pyspark.sql.functions import sum
#Group By and Aggregate
#Calculate total revenue for each l_shipmode:

df_grouped = df_lineitem.groupBy("l_shipmode") \
    .agg((sum(df_lineitem.l_extendedprice * (1 - df_lineitem.l_discount))).alias("total_revenue"))
df_grouped.show()


+----------+-----------------+
|l_shipmode|    total_revenue|
+----------+-----------------+
|       AIR|155756263830.2007|
|      MAIL|155626832178.4431|
|      RAIL|155650102804.4504|
|      SHIP|155659562319.9524|
|     TRUCK|155703299295.9749|
|   REG AIR|155782357803.3855|
|       FOB|155656761014.8085|
+----------+-----------------+



In [0]:
#Add a New Column
df_with_total_price = df_lineitem.withColumn("total_price", df_lineitem.l_extendedprice * (1 - df_lineitem.l_discount))
df_with_total_price.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+-----------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|           l_comment|total_price|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+-----------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|   REG AIR| even pinto beans...| 65850.8400|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-05-31|  1994-06-20| 

In [0]:
#Sort Data
df_sorted = df_lineitem.orderBy("l_quantity", ascending=False)
df_sorted.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+
|  15998113|   943289|     5808|           2|     50.00|       66612.00|      0.08| 0.02|           N|           O|1998-09-24|  1998-08-18|   1998-10-02|TAKE BACK RETURN|       AIR|al packages. blit...|
|  25245639|   336791|    36792|           3|     50.00|       91389.00|      0.10| 0.07|           N|           O|1995-12-03|  1996-01-09|   1995-12-22|            NONE|      MAIL|ly fina

In [0]:
#Drop a Column
df_dropped = df_lineitem.drop("l_comment")
df_dropped.show(5)

+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|   REG AIR|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-05-31|  1994-06-20|   1994-06-22|            NONE|       FOB|
|  15997988|   904286|     4287|           2|     37.00|       47738.88|      0.07| 0.05| 

In [0]:
#Rename a Column
#Rename the l_shipmode column to shipping_mode:
df_renamed = df_lineitem.withColumnRenamed("l_shipmode", "shipping_mode")
df_renamed.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+-------------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|shipping_mode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+-------------+--------------------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|      REG AIR| even pinto beans...|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-05-31|  1994-06-20|   1994-06-22|            NONE|      

In [0]:
#Filter Rows Using Multiple Conditions

df_filtered_multi = df_lineitem.filter((df_lineitem.l_shipmode == "AIR") & (df_lineitem.l_quantity > 30))
df_filtered_multi.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|   l_shipinstruct|l_shipmode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+-----------------+----------+--------------------+
|  15997989|   150305|      306|           2|     49.00|       66409.70|      0.01| 0.08|           R|           F|1993-01-18|  1993-02-05|   1993-02-06|             NONE|       AIR|       ometimes iron|
|  15998018|   650130|    12644|           1|     50.00|       54005.00|      0.06| 0.07|           N|           O|1997-10-20|  1997-10-09|   1997-10-28|      COLLECT COD|       AIR|th

In [0]:
#Calculate Discounted Price
df_discounted_price = df_lineitem.withColumn(
    "discounted_price", 
    df_lineitem.l_extendedprice * (1 - df_lineitem.l_discount)
)
df_discounted_price.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+----------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|  l_shipinstruct|l_shipmode|           l_comment|discounted_price|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+----------------+----------+--------------------+----------------+
|  15997987|   295335|    20346|           4|     50.00|       66516.00|      0.01| 0.08|           A|           F|1992-02-12|  1992-04-22|   1992-03-10|TAKE BACK RETURN|   REG AIR| even pinto beans...|      65850.8400|
|  15997988|   332059|    19578|           1|     49.00|       53460.96|      0.08| 0.03|           A|           F|1994-

In [0]:
#Filter Based on Date Range
from pyspark.sql.functions import col

df_date_filtered = df_lineitem.filter(
    (col("l_shipdate") >= "1995-01-01") & (col("l_shipdate") <= "1995-12-31")
)
df_date_filtered.show(5)


+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------+----------+--------------------+
|l_orderkey|l_partkey|l_suppkey|l_linenumber|l_quantity|l_extendedprice|l_discount|l_tax|l_returnflag|l_linestatus|l_shipdate|l_commitdate|l_receiptdate|l_shipinstruct|l_shipmode|           l_comment|
+----------+---------+---------+------------+----------+---------------+----------+-----+------------+------------+----------+------------+-------------+--------------+----------+--------------------+
|  15997991|   466072|     3600|           1|     26.00|       26989.30|      0.04| 0.00|           N|           F|1995-05-31|  1995-07-11|   1995-06-30|   COLLECT COD|     TRUCK|          es. regula|
|  15997991|   271922|    34428|           2|     26.00|       49241.66|      0.01| 0.05|           N|           O|1995-06-19|  1995-06-23|   1995-07-06|   COLLECT COD|       AIR|ages are fluffily

In [0]:
#Bar Chart: Display Ship Mode and count
display(df_lineitem.select("l_shipmode"))

l_shipmode
REG AIR
FOB
RAIL
FOB
RAIL
TRUCK
RAIL
AIR
AIR
MAIL


Databricks visualization. Run in Databricks to view.

In [0]:
#Pie Chart: Proportion of Orders by Return Flag
from pyspark.sql.functions import count

df_return_flag = df_lineitem.groupBy("l_returnflag") \
    .agg(count("*").alias("order_count"))

display(df_return_flag)


l_returnflag,order_count
A,7403889
N,15189553
R,7406353


Databricks visualization. Run in Databricks to view.

In [0]:
#Line Chart: Average Quantity Over Time
from pyspark.sql.functions import avg

df_avg_quantity = df_lineitem.groupBy("l_shipdate") \
    .agg(avg("l_quantity").alias("avg_quantity")) \
    .orderBy("l_shipdate")

display(df_avg_quantity)

l_shipdate,avg_quantity
1992-01-02,26.467391
1992-01-03,26.331707
1992-01-04,24.792982
1992-01-05,25.588235
1992-01-06,24.737037
1992-01-07,24.981356
1992-01-08,25.756241
1992-01-09,24.872538
1992-01-10,25.409692
1992-01-11,25.537583


Databricks visualization. Run in Databricks to view.

In [0]:
#Histogram: Distribution of Quantity
display(df_lineitem.select("l_quantity"))

l_quantity
50.00
49.00
37.00
13.00
29.00
13.00
15.00
49.00
19.00
4.00


Databricks visualization. Run in Databricks to view.

In [0]:
#Scatter Plot: Quantity vs. Extended Price
display(df_lineitem.select("l_quantity", "l_extendedprice"))


l_quantity,l_extendedprice
50.00,66516.00
49.00,53460.96
37.00,47738.88
13.00,25036.44
29.00,34744.32
13.00,19883.63
15.00,24368.70
49.00,66409.70
19.00,23182.85
4.00,5132.64


Databricks visualization. Run in Databricks to view.